In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import rv_continuous
from scipy.special import gamma, comb, beta
from itertools import product
import time
import matplotlib.pyplot as plt

In [ ]:
# Inverse CDF for e^-|t|
def inv_cdf(x):
    if x >= 1/2:
        return -np.log(1-2*(x-1/2))
    else:
        return np.log(2*x)

# Generate g vector
def gen_g(n):
    g = []
    u = np.random.uniform(0,1,n)
    for u_i in u:
        g_i = inv_cdf(u_i)
        g.append(g_i)
    return g

# Generate Z
def gen_Z():
    return np.random.exponential(1, 1)

# Generate random point in the diamond
def gen_point_in_diamond(n):
    g = np.array(gen_g(n))
    z = gen_Z()[0]
    x = []
    for g_i in g:
        x_i = g_i / (np.sum(abs(g)) + z)
        x.append(x_i)

    # Normalize volume
    x = np.array(x) * (gamma(1+n))**(1/n)/2

    return x

# Generate random point in the cube
def gen_point_in_cube(dimension, bounds=[-1/2, 1/2]):
    x = []
    for i in range(dimension):
        x_i = np.random.uniform(bounds[0], bounds[1])
        x.append(x_i)
    return np.array(x)

In [ ]:
def get_pairwise_distances(points):
    m = len(points)
    pairwise_distances = np.zeros(shape=(m, m))
    upper_tri_indices = np.triu_indices(m, k=1)

    for i, j in zip(upper_tri_indices[0], upper_tri_indices[1]):
        pairwise_distances[i,j] = np.linalg.norm(points[i] - points[j])
        
    return pairwise_distances

In [ ]:
def check_for_contraction(pairwise_distances1, pairwise_distances2):
    upper_triangular = np.triu(np.ones_like(pairwise_distances1, dtype=bool),k=1)
    contraction = np.where(upper_triangular, pairwise_distances1 >= pairwise_distances2, True)
    if contraction.all():
        return True
    else:
        return False

In [ ]:
def create_grid(min_val, max_val, n0, n):
    num_step = int(np.ceil(n0**(1/n)))
    steps = np.linspace(min_val, max_val, num_step)
    permutation = list(product(steps, repeat=n))
    grid = np.array(permutation)
    return grid

In [ ]:
def estimate_volume_of_intersection_grid(centers, n, r, n0=100000):
    n0 = 20**n
    all_points = np.concatenate(centers)
    all_points = np.abs(all_points)
    extreme_point = np.max(np.ceil(all_points))
    bounds = np.array([-(extreme_point+r), extreme_point+r])

    
    grid = create_grid(bounds[0], bounds[1], n0, n)
    
    intersection = sum(all(np.linalg.norm(sample - center) <= r for center in centers) for sample in grid)
    volume_box = (bounds[1] - bounds[0])**n
    volume = (intersection / n0) * volume_box
    
    return volume

In [ ]:
def find_contraction(p, n, r, N):
    # Time how long it takes
    start = time.time()

    # Generate random points
    points_in_cube = []
    points_in_diamond = []
    for i in range(N):
        point_i = gen_point_in_cube(n)
        points_in_cube.append(point_i)
        point_j = gen_point_in_diamond(n)
        points_in_diamond.append(point_j)

    # Get pairwise distances and averages
    pairwise_distances_cube = get_pairwise_distances(points_in_cube)
    pairwise_distances_diamond = get_pairwise_distances(points_in_diamond)
    largest_avg_pairwise_distance = np.average(pairwise_distances_cube)
    smallest_avg_pairwise_distance = np.average(pairwise_distances_diamond)
    
    k = 1
    # Check for contraction
    if check_for_contraction(pairwise_distances_cube, pairwise_distances_diamond):
            cube_vol = estimate_volume_of_intersection_grid(points_in_cube, n, r)
            diamond_vol = estimate_volume_of_intersection_grid(points_in_diamond, n, r)
            end = time.time()
            return k, points_in_cube, points_in_diamond, pairwise_distances_cube, pairwise_distances_diamond, cube_vol, diamond_vol, start, end

    # Continue to generate new points while updated the largest and smallest pairwise distance averages
    while True:
        if k % 100 == 0:
            print(k)
        new_points_in_diamond = []
        new_points_in_cube = []
        for i in range(N):
            point_i = gen_point_in_cube(n)
            new_points_in_cube.append(point_i)
            point_j = gen_point_in_diamond(n)
            new_points_in_diamond.append(point_j)

        new_pairwise_distances_cube = get_pairwise_distances(new_points_in_cube)
        new_pairwise_distances_diamond = get_pairwise_distances(new_points_in_diamond)

        # Update largest
        cube_average = np.average(new_pairwise_distances_cube)
        if cube_average > largest_avg_pairwise_distance:
            points_in_cube = new_points_in_cube
            largest_avg_pairwise_distance = cube_average
            pairwise_distances_cube = new_pairwise_distances_cube

        # Update smallest
        diamond_average = np.average(new_pairwise_distances_diamond)
        if diamond_average < smallest_avg_pairwise_distance:
            points_in_diamond = new_points_in_diamond
            smallest_avg_pairwise_distance = diamond_average
            pairwise_distances_diamond = new_pairwise_distances_diamond

        
        
        k += 1
        
        # Check for contraction
        if check_for_contraction(pairwise_distances_cube, pairwise_distances_diamond):
                cube_vol = estimate_volume_of_intersection_grid(points_in_cube, n, r)
                diamond_vol = estimate_volume_of_intersection_grid(points_in_diamond, n, r)
                end = time.time()
                return k, points_in_cube, points_in_diamond, pairwise_distances_cube, pairwise_distances_diamond, cube_vol, diamond_vol, start, end
        


In [ ]:
# compute average pairwise distances
n = 50
N = 1000
cube = []
for i in range(N):
    cube.append(gen_point_in_cube(n))
matrix_c = get_pairwise_distances(cube)
ave_c = np.mean(matrix_c[matrix_c != 0])
print(f"Cube Average: {ave_c}")

diamond = []
for i in range(N):
    diamond.append(gen_point_in_diamond(n))
matrix_d = get_pairwise_distances(diamond)
ave_d = np.mean(matrix_d[matrix_d != 0])
print(f"Diamond Average: {ave_d}")

Monte Carlo Integration:$\\$
$\displaystyle \int_K f(x)dx \approx \frac{Vol(K)}{N} \sum_{i=1}^N f(\bar{x_i})\\$ as $N \rightarrow \infty$

In [ ]:
# estimate isotropic constants with monte carlo integration
iso = pd.DataFrame(columns=['n', 'L_Q', 'L_C'])
k = 0
for n in range(10, 301, 20):
    N = 1000
    cube = []
    for i in range(N):
        pt = gen_point_in_cube(n)
        cube.append((np.linalg.norm(pt)**2))
    integral_c = np.sum(cube)/N

    diamond = []
    for i in range(N):
        pt = gen_point_in_diamond(n)
        diamond.append((np.linalg.norm(pt)**2))
    integral_d = np.sum(diamond)/N

    iso.loc[k] = [n, (integral_c/n)**(1/2), (integral_d/n)**(1/2)]
    k += 1
iso_tex = iso.to_latex(index=False)
print(iso)
print(iso_tex)


In [ ]:
# Computing c_n in L_{C_n}
for n in range(1,152,10):
    print(n, (n*beta(3,n))**(1/2) * (gamma(1+n))**(1/n)/2)

# Calculate expected volume of intersection

In [ ]:
n = 6
r = n**(1/2)
N = 16
num_iter = 30

table = pd.DataFrame(columns=["Cube Vol", "Cube pts", "Cube ||.||_2 Average", "Diamond Vol", "Diamond pts", "Diamond ||.||_2 Average"])
for i in range(num_iter):
    cube = []
    diamond = []
    for j in range(N):
        cube.append(gen_point_in_cube(n))
        diamond.append(gen_point_in_diamond(n))

    cube_dist = get_pairwise_distances(cube)
    cube_ave_dist = np.mean(cube_dist[cube_dist != 0])
    diamond_dist = get_pairwise_distances(diamond)
    diamond_ave_dist = np.mean(diamond_dist[diamond_dist != 0])

    cube_vol = estimate_volume_of_intersection_grid(cube, n, r)
    diamond_vol = estimate_volume_of_intersection_grid(diamond, n, r)

    table.loc[i] = [cube_vol, cube, cube_ave_dist, diamond_vol, diamond, diamond_ave_dist]

print(table[['Cube Vol', "Cube ||.||_2 Average", 'Diamond Vol', "Diamond ||.||_2 Average"]])
print(f"E: {round(np.mean(table['Cube Vol']),6)}\t {round(np.mean(table['Diamond Vol']),6)}")

In [ ]:
print(table[['Cube Vol', "Cube ||.||_2 Average", 'Diamond Vol', "Diamond ||.||_2 Average"]].to_latex(index=True))

In [ ]:
print(np.mean(table["Diamond ||.||_2 Average"]))

In [ ]:
for i in range(table.shape[0]):
    for j in range(table.shape[0]):
        if table.loc[i,'Diamond Vol'] < table.loc[j, 'Cube Vol']:
            pwd_cube = get_pairwise_distances(table.loc[j, "Cube pts"])
            pwd_diamond = get_pairwise_distances(table.loc[i, "Diamond pts"])
            if check_for_contraction(pwd_cube, pwd_diamond):
                print(f"Contraction found for diamond index {i} and cube index {j}") 